In [1]:
import os, re, string
from collections import defaultdict


def pulizia(s):
    s = s.replace("\n", "")
    s = s.replace("a'", "à")
    s = s.replace("e'", "è")
    s = s.replace("i'", "ì")
    s = s.replace("o'", "ò")
    s = s.replace("u'", "ù")
    s = re.sub(r'(?<=[.,?!\'])(?=[^\s])', r' ', s)
    s = s.translate(str.maketrans('', '', string.punctuation))
    return s


d = {}
d2 = defaultdict(list)

for filename in os.listdir('./Corpus/'):
    if filename.endswith('.txt'):
        with open(os.path.join('./Corpus/', filename)) as f:
            
            content = f.read()
            normalized = content.lower()
            normalized = pulizia(normalized)
            
            filename = filename.split(".")[0]
            year, pres = (filename.split("-"))
            year = int(year)
            
        d[year] = {"presidente":pres, "lunghezza":len(content.split()), "discorso_norm":normalized, "discorso_orig":content}
        d2[pres].append(normalized)
        
d3 = {}
for pres,los in d2.items():
    d3[pres] = {"discorso_norm":" ".join(los)}
d2 = None

In [2]:
from collections import Counter
import nltk
from nltk.corpus import stopwords

ignored_words = nltk.corpus.stopwords.words('italian')

def remove_sw(diz, ignored_words=ignored_words):
    return [w for w in diz["discorso_norm"].split() if w not in ignored_words]

for year, diz in d.items():
    tokens = remove_sw(diz)
    d[year]["most_common"] = {k:v for k,v in Counter(tokens).most_common(30)}

In [3]:
from nltk.stem import SnowballStemmer

stemmer_snowball = SnowballStemmer('italian')    
    
for pres, diz in d3.items():
    tokens = remove_sw(diz)
    tokens = [stemmer_snowball.stem(w) for w in tokens]
    d3[pres]["most_common"] = {k:v for k,v in Counter(tokens).most_common(30)}

In [4]:
import spacy
nlp = spacy.load('it_core_news_sm')

def gulp(n_chars: int, n_words: int, n_sents: int) -> float:
    return (300 * n_sents / n_words) - (10 * n_chars / n_words) + 89

for year, diz in d.items():
    
    n_chars = len(diz["discorso_orig"])
    speech = nlp(diz['discorso_orig'])
    n_words = len(speech)
    n_sents = 0
    for sent in speech.sents:
        n_sents += 1
    
    tmp = []
    
    for token in speech:
        tmp.append(token.pos_)
    
    counter = dict(Counter(tmp)).items()
    len_per = {k:(v/d[year]["lunghezza"])*100 for k,v in counter}
    d[year]["PoS_percentage"] = {k:v for k,v in sorted(len_per.items(),key=lambda i:i[1], reverse=True)}
    d[year]["indice_leggibilita"] = gulp(n_chars, n_words, n_sents)

In [5]:
import json

with open('corpus_xdata.json', 'w') as f:
    json.dump(d, f)
    
with open('corpus_xpresidente.json', 'w') as f:
    json.dump(d3, f)

In [6]:
# todos

# lower / capitalize speeches in capslock
# trattini e punteggiatura speciale: verificare
# collocazioni